In [3]:
!unzip dataset.jsonl.zip

Archive:  dataset.jsonl.zip
  inflating: dataset.jsonl           
  inflating: __MACOSX/._dataset.jsonl  


In [9]:
import requests
import pandas as pd
import json
import time

In [4]:
# Парсинг jsonl

def read_jsonl(filename):
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

def process_data(data):
    posts = []
    comments = []
    for el in data:
        if 'root_id' not in el:
            posts.append(el)
        else:
            comments.append(el)
    return posts, comments

posts, comments = process_data(read_jsonl('../input/dataset.jsonl'))

# pd.DataFrame(posts).to_pickle('posts.pkl')
# pd.DataFrame(comments).to_pickle('comments.pkl')
posts = pd.DataFrame(posts)
comments = pd.DataFrame(comments)

comments['date'] = pd.to_datetime(comments['date'], unit='s')
comments.sort_values(by='date', inplace=True)

In [5]:
# Подготовка текстов комментов

N = posts.shape[0] # posts.shape[0]
SEQ_LEN = 1500

posts_idx = []
posts_hashes = []
comments_hashes = []
texts = []

for index, row in posts[0:N].iterrows():
    id = row['id']
    parent_hash = row['hash']
    posts_idx.append(id)
    posts_hashes.append(parent_hash)

    sl = comments.loc[comments.parent_id==id]
    text = ' '.join(sl.text.values)[0:SEQ_LEN]
    childrens_hashes = list(sl.hash.values)
    comments_hashes.append(childrens_hashes)
    texts.append(text)

In [6]:
len(texts)

10008

In [7]:
texts[1000]

'После прослушивания записи сижу просто в шоке! После 3-4 сессий терапии должно стать легче?! Я чувствую себя просто тупой дурой! Прошла Машин курс по психосоматике, плюс личная терапия с другими психологами, плюс терапевтическая группа  год. Плюс 7 лет работала по системе Д. Троцкого. И ничего! Ребята! О! Что еще надо сделать чтобы болезнь сдвинулась с места.? Я сделала все, что было в курсе. В реальной жизни тоже сделала, то что было необходимо. Как в песок! Что не так? Помогите! Питание, БАДы. Режим. Все есть. Подключилось еще сердце. Прям организм мне орет! Не туда идешь. Никаких улучшений. Ну должно же было хоть сто то измениться? Ну хоть на 10%? Хоть на 5? Или есть такие пациенты в вашей практике? Которым не помогает? Ну тоже психолог, который основывается на знания, которым не менее 2000 лет. Ну и на всех религиях мира. Т.е. его система не противоречит ни одному духовному закону и ни одной религии.'

In [69]:
answers = []
inputs = []
iam_token = '''xxxxxxxxxxxx'''
x_folder_id = 'xxxxxxxxxxxx'

In [70]:
instruction = """
На основе этих комментариев дай сокращенное предложение (summary) о том, что обсуждается в этом тексте.
Напиши только 1-3 предложения.
Выделите основные концепции и идеи, используя не более 3 предложений.
Обобщи содержание в нескольких предложениях, не теряя смысла.
Предоставь в ответе краткое содержание текста с сохранением смысла. Имей ввиду, что этот текст == это комментарии из социальных сетей.
Сфокусируйся на основных идеях и выводах, не углубляясь в детали.
"""
# Выделите основные концепции и идеи, используя не более 3 предложений.
# Обобщи содержание в нескольких предложениях, не теряя смысла.
# Предоставь в ответе краткое содержание текста с сохранением смысла. Имей ввиду, что этот текст == это комментарии из социальных сетей.
# Сфокусируйся на основных идеях и выводах, не углубляясь в детали.

# НЕ ИСПОЛЬЗУЙ подводок в духе: 
# В этом тексте обсуждается...
# В комментариях под постом обсуждаются...
# Пост о том, как...
# Люди обсуждают...
# и им подобных.
# Сразу выдавай основную мысль.
# Если в ответе есть вводная часть типа В этом тексте обсуждается..., В комментариях под постом обсуждаются..., Пост о том, как..., Люди обсуждают..., В комментариях к посту...
# то удаляй эту вводную часть.

for index, text in enumerate(texts):
        
    error = 'ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed: allowed 3 requests'
    
    try:
        result = requests.post(
        url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
        # headers={'Authorization': f'Bearer {iam_token}', 'x-folder-id': x_folder_id},
        headers={"Authorization": "Api-Key xxxxxxxxxxxx"},
        json={
          "model": "general",
          "instruction_text": "Ниже идет множество комментариев под постом, собранных вместе:\n\n",
          "request_text": f"{text}\n\n{instruction}",
          "generation_options": {
            "max_tokens": 1500,  
            "temperature": 0.5
              }
            }
        )
        answer = result.json()['result']['alternatives'][0]['text']
        error = '-'
    except Exception as e:
        time.sleep(1)
        error = result.json()['error']['message']
        # print(error)
        answer = '-'
        while 'ai.textGenerationCompletionSessionsCount.count gauge quota limit exceed:' in error:
            time.sleep(1)
            try:
                result = requests.post(
                url='https://llm.api.cloud.yandex.net/llm/v1alpha/instruct',
                # headers={'Authorization': f'Bearer {iam_token}', 'x-folder-id': x_folder_id},
                headers={"Authorization": "Api-Key xxxxxxxxxxxx"},
                json={
                  "model": "general",
                  "instruction_text": "Ниже идет множество комментариев под постом, собранных вместе:\n\n",
                  "request_text": f"{text}\n\n{instruction}",
                  "generation_options": {
                    "max_tokens": 1500,  
                    "temperature": 0.5
                      }
                    }
                )
                answer = result.json()['result']['alternatives'][0]['text']
                error = '-'
            except Exception as e:
                time.sleep(1)
                error = result.json()['error']['message']
                # print(error)
                answer = '-'

        
    answers.append(answer)
    inputs.append(text)
    
    if index%100 == 0:
        print(index)
        df = pd.DataFrame({"text": inputs, "answer": answers})
        df.to_excel('df_result_final.xlsx', index=False)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000


In [71]:
index

10007

In [72]:
df = pd.DataFrame({"text": inputs, "answer": answers})
# df.to_csv('df_result.csv', index=False)

In [73]:
df

,text,answer
0,"Хорошая Лерочка ты женщина, хозяюшка и заботл...",Текст представляет собой набор комментариев по...
1,Лапкозаврик встраивает узел в пустую ячейку и ...,"В посте обсуждаются различные аспекты игры ""Ла..."
2,интересно. эмпирическим методом надо это прове...,Комментарии обсуждают рацион правильного питан...
3,"Юрий хороший вы человек, да поможет вам Бог .❤...","В этом комментарии обсуждается, что ролики Юри..."
4,Вау... прекрасно ❤ Как всегда идеально❤ Атмосф...,В комментариях к посту автор выражает свою рад...
...,...,...
10003,🙏🙏🙏 🙏🙏🙏🌱❤🌹🌹🌹🌹🌹 🙏🙏🙏 🙏🙏🙏🙏🙏❤ Аминь🙏🙏🙏❤❤❤🌷🌷🌷 🙏🙏🙏 🙏...,В данном тексте обсуждаются комментарии под по...
10004,"Ух ты, здорово Участвую Будем участвовать Как ...",В этом посте обсуждаются разные мнения и точки...
10005,Банда зигзага Слово пацана Слово пацана Слово ...,В комментариях к посту в социальных сетях обсу...
10006,"Напоминаю о том, что делает нашу жизнь гармони...",Вот основные идеи и ключевые моменты из текста...


In [74]:
df.to_excel('df_result_final.xlsx', index=False)